In [1]:
import skmob

In [2]:
!pip install psycopg2-binary pandas sqlalchemy

In [3]:
import psycopg2 as pg
import pandas as pd
import numpy as np

In [4]:
dbprops = {
    'host': 'dbapp.hanyang.ac.kr',
    'database': 'testbed-jgs',
    'user': 'postgres',
    'password': '1234',
    'port': '54321'
}
conn = pg.connect(**dbprops)

In [5]:
cur = conn.cursor()

In [6]:
sql = "select tp.fixtime at time zone 'utc' at time zone 'Asia/Seoul' as datetime, tp.* from tc_devices td, tc_positions tp where td.id = 1 AND tp.deviceid = td.id AND tp.deviceid = td.id AND tp.fixtime BETWEEN '2020-10-22 09:00:00'::timestamp AND '2020-10-23 02:00:00'::timestamp ORDER BY tp.id"
df = pd.read_sql(sql, conn)
df.head()

,datetime,id,protocol,deviceid,servertime,devicetime,fixtime,valid,latitude,longitude,altitude,speed,course,address,attributes,accuracy,network
0,2020-10-22 18:00:19,441010,osmand,1,2020-10-22 09:00:21.240,2020-10-22 09:00:19,2020-10-22 09:00:19,True,37.555743,127.049386,64.400002,0.000000,0.000000,None,"{""batteryLevel"":92.0,""distance"":0.81,""totalDis...",13.698,null
1,2020-10-22 18:10:20,441011,osmand,1,2020-10-22 09:10:22.072,2020-10-22 09:10:20,2020-10-22 09:10:20,True,37.555755,127.049387,63.199997,0.000000,0.000000,None,"{""batteryLevel"":96.0,""distance"":1.3,""totalDist...",14.070,null
2,2020-10-22 18:28:30,441014,osmand,1,2020-10-22 09:28:31.918,2020-10-22 09:28:30,2020-10-22 09:28:30,True,37.555747,127.049387,63.199997,0.000000,0.000000,None,"{""batteryLevel"":100.0,""distance"":0.88,""totalDi...",13.488,null
3,2020-10-22 18:31:01,441015,osmand,1,2020-10-22 09:31:03.230,2020-10-22 09:31:01,2020-10-22 09:31:01,True,37.555854,127.049336,64.599998,0.000000,0.000000,None,"{""batteryLevel"":100.0,""distance"":12.67,""totalD...",11.510,null
4,2020-10-22 18:31:23,441016,osmand,1,2020-10-22 09:31:25.213,2020-10-22 09:31:23,2020-10-22 09:31:23,True,37.555838,127.049337,64.599998,0.093432,147.637253,None,"{""batteryLevel"":100.0,""distance"":1.71,""totalDi...",11.042,null


In [7]:
df2=df[['datetime', 'latitude', 'longitude', 'altitude', 'speed', 'course', 'address', 'attributes']]
df2.head()
#irr1=df2.iloc[100:115, :]
#irr2=df2.iloc[113:115, :]
#re1=df2.iloc[116:120,:]

,datetime,latitude,longitude,altitude,speed,course,address,attributes
0,2020-10-22 18:00:19,37.555743,127.049386,64.400002,0.000000,0.000000,None,"{""batteryLevel"":92.0,""distance"":0.81,""totalDis..."
1,2020-10-22 18:10:20,37.555755,127.049387,63.199997,0.000000,0.000000,None,"{""batteryLevel"":96.0,""distance"":1.3,""totalDist..."
2,2020-10-22 18:28:30,37.555747,127.049387,63.199997,0.000000,0.000000,None,"{""batteryLevel"":100.0,""distance"":0.88,""totalDi..."
3,2020-10-22 18:31:01,37.555854,127.049336,64.599998,0.000000,0.000000,None,"{""batteryLevel"":100.0,""distance"":12.67,""totalD..."
4,2020-10-22 18:31:23,37.555838,127.049337,64.599998,0.093432,147.637253,None,"{""batteryLevel"":100.0,""distance"":1.71,""totalDi..."


## speed가 0이상일 때만 출력

In [8]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# SpeedOverZero
soz = df2[df2['speed'] > 0]
soz
soz.to_csv('SpeedOverZero.csv', header=True, index=True)

In [9]:
tdf=skmob.TrajDataFrame(df2, latitude='latitude', longitude='longitude', datetime='datetime', user_id='deviceid')
tdf2=skmob.TrajDataFrame(soz, latitude='latitude', longitude='longitude', datetime='datetime', user_id='device_id')

#### TDF만 지도위에 시각화

In [10]:
# tdf
# tdf.plot_trajectory()
tdf2
tdf2.plot_trajectory()

#### TDF를 parameter를 줘서 시각화

In [11]:
from skmob.preprocessing import filtering
ftdf = filtering.filter(tdf2, max_speed_kmh=300, include_loops=True, max_loop=6, ratio_max=0.5)
ftdf.plot_trajectory()

#### Stop지점만 Detected

In [12]:
from skmob.preprocessing import detection
stdf = detection.stops(ftdf, stop_radius_factor=0.5, minutes_for_a_stop=20.0, spatial_radius_km=0.2, leaving_time=True)
stdf.plot_stops()

## speed가 0일때 

In [14]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# SpeedEqualZero
sez = df2[df2['speed'] == 0]
sez

sez.to_csv('SpeedEqualZero.csv', header=True, index=True)

In [15]:
#tdf=skmob.TrajDataFrame(df2, latitude='latitude', longitude='longitude', datetime='datetime', user_id='deviceid')
#tdf2=skmob.TrajDataFrame(soz, latitude='latitude', longitude='longitude', datetime='datetime', user_id='device_id')
tdf3=skmob.TrajDataFrame(sez, latitude='latitude', longitude='longitude', datetime='datetime', user_id='deviceid')

In [16]:
tdf3
tdf3.plot_trajectory()

## 이게 왠지 더 튀는듯...

In [17]:
from skmob.preprocessing import filtering
ftdf2 = filtering.filter(tdf3, max_speed_kmh=300, include_loops=True, max_loop=6, ratio_max=0.5)
ftdf2.plot_trajectory()

In [18]:
from skmob.preprocessing import detection
stdf2 = detection.stops(ftdf2, stop_radius_factor=0.5, minutes_for_a_stop=20.0, spatial_radius_km=0.2, leaving_time=True)
stdf2.plot_stops()